In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
import os
import pickle
from matplotlib.font_manager import FontProperties
sns.set(font=['sans-serif'])
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})
# scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer, RobustScaler
from sklearn import datasets, linear_model, preprocessing, neighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix,mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold,RandomizedSearchCV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
import xgboost as xgb

In [2]:
df_oh = pd.read_csv('../台中市_predictset_onehot.csv', engine='python',index_col=[0])
df_oh

,交易年,總樓層數,主要建材,有無管理組織,建物移轉總面積坪,每坪價格,車位,屋齡,房間數,樓層比,...,交易標的_房地(土地+建物),交易標的_房地(土地+建物)+車位,都市土地使用分區_住,都市土地使用分區_其他,都市土地使用分區_商,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_套房(1房1廳1衛),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
0,7,3,4,0,19.462850,16.698479,0,45,6,1.000000,...,1,0,1,0,0,0,1,0,0,0
1,7,7,4,1,8.929800,16.797689,0,26,3,0.571429,...,1,0,1,0,0,0,0,0,1,0
2,7,7,4,1,28.716325,18.630518,0,26,7,0.285714,...,1,0,1,0,0,0,0,0,1,0
3,7,14,4,1,31.133300,19.271969,1,19,5,0.642857,...,0,1,1,0,0,1,0,0,0,0
4,7,5,4,0,22.669350,13.233727,0,40,6,1.000000,...,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194344,9,24,4,0,23.232000,37.017906,0,0,5,0.333333,...,1,0,1,0,0,1,0,0,0,0
194345,9,24,4,0,48.109600,30.659162,1,0,7,0.500000,...,0,1,1,0,0,1,0,0,0,0
194346,9,24,4,0,29.490725,33.569877,1,0,5,0.833333,...,0,1,1,0,0,1,0,0,0,0
194347,9,24,4,0,47.453175,30.998980,1,0,7,0.541667,...,0,1,1,0,0,1,0,0,0,0


## XGBRegressor

In [9]:
X=df_oh.drop(columns=['每坪價格','車位'])
y=df_oh['每坪價格']

In [10]:
# 調參前
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 建立 XGBRegressor 模型
xgbr=xgb.XGBRegressor(tree_method="gpu_hist")
# xgbr=xgb.XGBClassifier(tree_method="auto",enable_categorical=True)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [11]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 2.920,測試集: 3.086
訓練集MAE: 2.068,測試集: 2.152
訓練集R2: 0.818,測試集: 0.795


In [ ]:
importance=xgbr.feature_importances_
importance

In [ ]:
X_del=pd.DataFrame({'importance':xgbr.feature_importances_,'features':X.columns}).sort_values(by=['importance'],ascending=False)
mask_selected=X_del.iloc[0:20].features
X_selected=X_train[list(mask_selected)]
X_selected

In [ ]:
# grid 調參

params = {'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
              }

xgbclf = xgb.XGBRegressor(tree_method='gpu_hist',enable_categorical=True)
clf = RandomizedSearchCV(estimator=xgbclf,
                         param_distributions=params,
                         n_iter=10,
                         n_jobs=-1,
                         cv=3,
                         verbose=1)

clf.fit(X_train, y_train)

print(clf.best_params_,clf.best_score_)

In [12]:
# 調參後
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)

xgbr=xgb.XGBRegressor(tree_method="gpu_hist",enable_categorical=True,n_jobs=-1,
                      colsample_bytree=0.5,
                      min_child_weight=5,
                      subsample=0.7,
                      max_depth=11,
                      n_estimators=500,
                      learning_rate=0.03)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

# X_test = scaler.transform(X_test)
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [13]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 2.396,測試集: 2.841
訓練集MAE: 1.673,測試集: 1.915
訓練集R2: 0.877,測試集: 0.826


In [ ]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)
sc=cross_val_score(model,X, y, cv= kf)

In [14]:
# Saving model
with open("model_XGBR_taichung.pickle","wb") as f:
    pickle.dump(xgbr, f, protocol=pickle.HIGHEST_PROTOCOL)